# Data Extraction

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import praw
from psaw import PushshiftAPI
from datetime import datetime as dt
import time

In [9]:
client_id = "IHRxOIozokYZBI-72YpKfw"
client_secret = "a19KLEDV6oxrBZXz3FZ_h-2VtU_nIw"
user_agent = "Scraper 1.0 by /u/sc1015miniproject"

In [10]:
def setup_reddit_client(client_id, client_secret, user_agent):
    return praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)

def extract_comments_voo(reddit, subreddit_name, keyword, limit=1000):
    """
    Extract comments from the specified subreddit for submissions containing the keyword 'VOO'.
    Limit the number of comments to 1000.
    """
    subreddit = reddit.subreddit(subreddit_name)
    comments_data = []
    comment_count = 0

    for submission in subreddit.search(keyword, sort='new'):
        # Check if the submission title contains the keyword
        if keyword.lower() in submission.title.lower():
            submission.comments.replace_more(limit=None)
            for comment in submission.comments.list():
                if comment_count >= limit:
                    break

                if keyword.lower() in comment.body.lower():
                    comment_data = {
                        'author': str(comment.author),
                        'id': comment.id,
                        'created_utc': dt.fromtimestamp(comment.created_utc),
                        'permalink': comment.permalink,
                        'body': comment.body,
                        'score': comment.score,
                        'subreddit': str(comment.subreddit)
                    }
                    comments_data.append(comment_data)
                    comment_count += 1

        if comment_count >= limit:
            break

    return comments_data

In [11]:
# Initialize and extract comments
reddit = setup_reddit_client(client_id, client_secret, user_agent)
comments = extract_comments_voo(reddit, "ETFs", "VOO")
comments_voo_df = pd.DataFrame(comments)

In [12]:
comments_voo_df.head()

,author,id,created_utc,permalink,body,score,subreddit
0,coinslinger88,kxwqx9i,2024-04-04 04:28:47,/r/ETFs/comments/1buybes/sso_vs_spyvoo/kxwqx9i/,$VOO is for people who hate money,1,ETFs
1,coinslinger88,kxwr28w,2024-04-04 04:29:32,/r/ETFs/comments/1buvulh/just_starting_brokera...,$VOO is for homeless people,0,ETFs
2,Key-Mark4536,kxsmrdk,2024-04-03 10:17:45,/r/ETFs/comments/1buf5zo/voo/kxsmrdk/,"It's not a bad idea, but it wouldn’t add anyth...",6,ETFs
3,thefreewheeler,kxsjef1,2024-04-03 09:55:36,/r/ETFs/comments/1buf5zo/voo/kxsjef1/,No. VOO and VTI are nearly identical. Either a...,3,ETFs
4,Fun_Grapefruit_3416,kxwl2i1,2024-04-04 03:57:51,/r/ETFs/comments/1buf5zo/voo/kxwl2i1/,No. You already have VOO within VTI and the ta...,3,ETFs


In [13]:
comments_voo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   author       226 non-null    object        
 1   id           226 non-null    object        
 2   created_utc  226 non-null    datetime64[ns]
 3   permalink    226 non-null    object        
 4   body         226 non-null    object        
 5   score        226 non-null    int64         
 6   subreddit    226 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 12.5+ KB


### Save the DataFrame as a CSV File.

In [14]:
comments_voo_df.to_csv('../datasets/reddit_comment_voo.csv', index=False)